In [ ]:
import gradio as gr
import requests
import json

class TNBusAssistant:
    def __init__(self, api_key):
        self.api_key = api_key
        self.api_url = "https://api.groq.com/openai/v1/chat/completions"
        self.system_prompt = """You are a helpful Tamil Nadu Bus Route Assistant. Your ONLY purpose is to help users with:
- Bus routes in Tamil Nadu
- Bus timings and schedules
- Bus fare information
- Bus stand locations
- Government and private bus services in Tamil Nadu
- TNSTC (Tamil Nadu State Transport Corporation) information
- Route planning between Tamil Nadu cities

IMPORTANT RULES:
1. ONLY respond to questions related to Tamil Nadu bus routes, transportation, and travel
2. If asked about anything else (weather, food, movies, general knowledge, etc.), politely decline and say: "I'm specifically designed to assist with Tamil Nadu bus routes only. Please ask me about bus routes, timings, fares, or transportation in Tamil Nadu."
3. Be helpful, accurate, and friendly when discussing bus-related topics
4. If you don't have specific real-time information, provide general guidance and suggest contacting TNSTC or local bus stands

Always respond in a helpful manner for bus-related queries."""

    def is_bus_related_query(self, user_message):
        """Check if query is related to bus routes using Groq API"""
        check_prompt = f"""Analyze if this query is related to Tamil Nadu bus routes, transportation, or travel: "{user_message}"

Respond with ONLY "YES" or "NO".
- YES: if it's about buses, routes, transport, travel, timings, fares, TNSTC, bus stands in Tamil Nadu
- NO: if it's about anything else (weather, food, general chat, other topics)"""

        try:
            response = requests.post(
                self.api_url,
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": "llama-3.3-70b-versatile",
                    "messages": [{"role": "user", "content": check_prompt}],
                    "temperature": 0.1,
                    "max_tokens": 10
                },
                timeout=10
            )

            if response.status_code == 200:
                result = response.json()
                answer = result['choices'][0]['message']['content'].strip().upper()
                return "YES" in answer
            else:
                return True  # Default to allowing if check fails

        except Exception as e:
            print(f"Error in query check: {e}")
            return True  # Default to allowing if check fails

    def get_response(self, user_message, chat_history):
        """Get response from Groq API"""
        if not self.api_key or self.api_key.strip() == "":
            return "⚠️ Please enter your Groq API key first!"

        # Check if query is bus-related
        if not self.is_bus_related_query(user_message):
            return "🚌 I'm specifically designed to assist with Tamil Nadu bus routes only. Please ask me about bus routes, timings, fares, or transportation in Tamil Nadu."

        # Prepare messages
        messages = [{"role": "system", "content": self.system_prompt}]

        # Add chat history
        for human, assistant in chat_history:
            if human:
                messages.append({"role": "user", "content": human})
            if assistant:
                messages.append({"role": "assistant", "content": assistant})

        # Add current message
        messages.append({"role": "user", "content": user_message})

        try:
            response = requests.post(
                self.api_url,
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": "llama-3.3-70b-versatile",
                    "messages": messages,
                    "temperature": 0.7,
                    "max_tokens": 1024
                },
                timeout=30
            )

            if response.status_code == 200:
                result = response.json()
                return result['choices'][0]['message']['content']
            else:
                return f"❌ Error: {response.status_code} - {response.text}"

        except requests.exceptions.Timeout:
            return "⏱️ Request timed out. Please try again."
        except Exception as e:
            return f"❌ Error: {str(e)}"

# Global assistant instance
assistant = None

def initialize_assistant(api_key):
    """Initialize the assistant with API key"""
    global assistant
    if not api_key or api_key.strip() == "":
        return "⚠️ Please enter a valid Groq API key"
    assistant = TNBusAssistant(api_key)
    return "✅ API Key set successfully! You can now start chatting."

def chat(message, history, api_key):
    """Handle chat interactions"""
    global assistant

    if not api_key or api_key.strip() == "":
        return history + [[message, "⚠️ Please enter your Groq API key first!"]]

    if assistant is None or assistant.api_key != api_key:
        assistant = TNBusAssistant(api_key)

    response = assistant.get_response(message, history)
    history.append([message, response])
    return history

def clear_chat():
    """Clear chat history"""
    return []

# Create Gradio Interface
with gr.Blocks(title="Tamil Nadu Bus Route Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🚌 Tamil Nadu Bus Route Assistant
        ### Your AI-powered guide for Tamil Nadu bus routes, timings, and travel information

        Ask me about:
        - Bus routes between cities in Tamil Nadu
        - Bus timings and schedules
        - Fare information
        - TNSTC services
        - Bus stand locations
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            api_key_input = gr.Textbox(
                label="🔑 Groq API Key",
                placeholder="Enter your Groq API key here...",
                type="password"
            )
        with gr.Column(scale=1):
            api_status = gr.Textbox(
                label="Status",
                value="⏳ Waiting for API key...",
                interactive=False
            )

    set_key_btn = gr.Button("Set API Key", variant="primary")

    chatbot = gr.Chatbot(
        label="Chat",
        height=450,
        show_label=True,
        avatar_images=(None, "🚌")
    )

    with gr.Row():
        msg = gr.Textbox(
            label="Your Message",
            placeholder="Ask about Tamil Nadu bus routes, timings, fares...",
            scale=4
        )
        clear = gr.Button("Clear Chat", scale=1)

    gr.Examples(
        examples=[
            "What are the bus routes from Chennai to Madurai?",
            "How can I travel from Coimbatore to Ooty by bus?",
            "What is the fare for Chennai to Pondicherry?",
            "Tell me about TNSTC services",
            "Which buses go from Trichy to Thanjavur?"
        ],
        inputs=msg,
        label="Example Questions"
    )

    # Event handlers
    set_key_btn.click(
        fn=initialize_assistant,
        inputs=[api_key_input],
        outputs=[api_status]
    )

    msg.submit(
        fn=chat,
        inputs=[msg, chatbot, api_key_input],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )

    clear.click(fn=clear_chat, outputs=[chatbot])

    gr.Markdown(
        """
        ---
        **Note:** This chatbot only responds to questions about Tamil Nadu bus routes and transportation.
        Get your Groq API key from: [https://console.groq.com](https://console.groq.com)
        """
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(debug=True, share=True)

/tmp/ipython-input-2619786609.py:140: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Tamil Nadu Bus Route Assistant", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-2619786609.py:171: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2619786609.py:171: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f042266235287dd4dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
